In [18]:
from fforma.metrics import WeightedPinballLoss
from fforma.meta_learner import MetaLearnerNN
from fforma.meta_results_r_data import prepare_fforma_data

from ESRNN.m4_data import prepare_m4_data, seas_dict
from ESRNN.utils_evaluation import evaluate_prediction_owa

import pandas as pd

## Meta results from R (hyndman, et al)

In [10]:
feats_train, X_models_train, y_models_train, feats_test, X_models_test = prepare_fforma_data('data/hyndman_data', dataset_name='Hourly') #dataset_name=None for all data

Successfully decompressed  data/hyndman_data/raw/decompressed_data


## QFFORMA

In [11]:
nn_params = {'layers': [200, 100, 50, 25, 10], 
             'dropout': 0.1, 
             'epochs': 300, 
             'batch_size': 2,
             'learning_rate': 0.001, 
             'gradient_eps': 1e-8, 
             'weight_decay': 0,
             'freq_of_test': 10,
             'loss_function': WeightedPinballLoss(0.4)}

In [12]:
model = MetaLearnerNN(nn_params, X_models_train.set_index(['unique_id', 'ds']), y_models_train, 48)

In [13]:
model.fit(feats_train.set_index(['unique_id']), verbose_eval=False)

Testing loss: 0.57380
Testing loss: 0.08275
Testing loss: 0.06967
Testing loss: 0.06579
Testing loss: 0.05301
Testing loss: 0.06039
Testing loss: 0.05439
Testing loss: 0.06092
Testing loss: 0.05912
Testing loss: 0.05623
Testing loss: 0.05547
Testing loss: 0.05548
Testing loss: 0.06057
Testing loss: 0.05917
Testing loss: 0.05628
Testing loss: 0.05516
Testing loss: 0.06284
Testing loss: 0.05344
Testing loss: 0.06040
Testing loss: 0.05278
Testing loss: 0.06279
Testing loss: 0.06008
Testing loss: 0.05791
Testing loss: 0.05339
Testing loss: 0.05419
Testing loss: 0.05936
Testing loss: 0.05503
Testing loss: 0.05730
Testing loss: 0.05373
Testing loss: 0.06077


In [16]:
feats_test_id = feats_test.set_index('unique_id')
X_models_test_id = X_models_test.set_index(['unique_id', 'ds'])

In [19]:
fforma_predictions = model.predict(feats_test_id)
fforma_predictions = pd.DataFrame(fforma_predictions, 
                                  index=feats_test_id.index, 
                                  columns=X_models_test_id.columns)

In [20]:
preds = (fforma_predictions * X_models_test_id).sum(1)
preds = preds.rename('y_hat').to_frame().reset_index()

# Evaluation

In [25]:
#Freqs used by hyndman
freqs = {'Hourly': 24, 'Daily': 1,
         'Monthly': 12, 'Quarterly': 4,
         'Weekly':1, 'Yearly': 1}

In [26]:
def evaluate_fforma(dataset_name, fforma_df, directory, num_obs):
    _, y_train_df, X_test_df, y_test_df = prepare_m4_data(dataset_name=dataset_name,
                                                          directory=directory,
                                                          num_obs=num_obs)
    
    y_test_fforma = fforma_df[fforma_df['unique_id'].isin(y_test_df['unique_id'].unique())]
    y_test_fforma = y_test_fforma.rename(columns={'fforma_prediction': 'y_hat'})
    y_test_fforma = y_test_fforma.filter(items=['unique_id', 'ds', 'y_hat'])
    
    seasonality = freqs[dataset_name]
    owa, mase, smape = evaluate_prediction_owa(y_test_fforma, y_train_df, X_test_df, y_test_df, seasonality)
    
    return dataset_name, owa, mase, smape

In [27]:
evaluate_fforma('Hourly', preds, './data', 100_000)



===============  Model evaluation  ==============
OWA: 0.608 
SMAPE: 12.521 
MASE: 1.28 


('Hourly', 0.6076973863099461, 1.279661366283869, 12.520547504707347)